In [1]:
import threading, itertools, time, sys

class Signal : 
    go = True
    
def spin(msg, signal) : 
    write, flush = sys.stdout.write, sys.stdout.flush
    
    for char in itertools.cycle('|/-\\') : 
        status = char + ' ' + msg
        write(status)
        flush() 
        
        write('\x08' * len(status))
        time.sleep(.1)
        
        if not signal.go : 
            break
            
    write(' ' * len(status) + '\x08' * len(status))
    
def slow_function() : 
    time.sleep(3)
    return 42

def supervisor() : 
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object : ', spinner)
    
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    
    return result


def main() : 
    result = supervisor()
    print('Answer : ', result)
    
if __name__ == '__main__' : 
    main()

spinner object :  <Thread(Thread-5, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking          Answer :  42


In [20]:
"""
파이썬 버전 문제로 해당 부분 오류발생 
- Python 3.7부터 asyncio.async 함수가 더 이상 사용되지 않고, 대신 asyncio.create_task를 사용
- Python 3.4에서 도입된 @asyncio.coroutine 데코레이터와 yield from 구문도 Python 3.5 이상에서는 async와 await 키워드로 대체
"""
import asyncio, itertools, sys

@asyncio.coroutine
def spin(msg) : 
    write, flush = sys.stdout.write, sys.stdout.flush
    
    for char in itertools.cycle('|/-\\') : 
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try : 
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError : 
            break
            
    write(' ' * len(status) + '\x08' * len(status))
    
@asyncio.coroutine
def slow_function() : 
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor() : 
    spinner = asyncio.async(spin('thingking!'))
    print('spinner object : ', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main() : 
    loop = asyncio.get_event_loop()
    result = loop.run_untill_complete(supervisor())
    loop.close()
    print('Answer : ', result)
    
if __name__ == '__main__' : 
    main()

SyntaxError: invalid syntax (3729293308.py, line 31)

In [16]:
import sys
sys.version 


'3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0.0 ]'

In [21]:
import asyncio

def run_sync(coro_or_future) : 
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro_or_future)

a = run_sync(some_coroutine())

NameError: name 'some_coroutine' is not defined